<img align="left" src = logo-IJCLab-1.png width=250 style="padding: 10px"> 

**Notebook Author**: Martín Rodríguez Monroy

**Date last tested**: 2022-10-13

**Ran with RSP image**: Weekly 2022_40

**A large container is recommended for this notebook.**

This notebook demonstrates how to bin and do Principle Component Analysis (PCA) with survey property maps, and how to create a galaxy number map using the Object catalog.

This notebook builds off DP0.2 Tutorial Notebook 03c_Survey_Property_Maps.ipynb, available in the <a href="https://github.com/rubin-dp0/tutorial-notebooks">tutorial-notebooks repository</a>.
Data products are accessed through the Butler, and the user is expected to be familiar with the content of the introductory Butler tutorial in that repo (04a_Introduction_to_the_Butler.ipynb).

**Useful documentation**

<a href="https://healsparse.readthedocs.io/en/latest/index.html">HealSparse documentation</a>

<a href="https://buildmedia.readthedocs.org/media/pdf/python-for-multivariate-analysis/latest/python-for-multivariate-analysis.pdf">Principal Component Analysis notes</a>

**Credit:** Developed by Martín Rodríguez Monroy with the help and editing of Sylvie Dagoret-Campagne and Melissa Graham. This notebook is based in part on material originally developed by Eli Rykoff for the DP0.2 Tutorial Notebook 03c_Survey_Property_Maps.ipynb


# 1.0. Import packages

In [ ]:
# general python packages
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import ZScaleInterval, LinearStretch, ImageNormalize
from astropy.wcs import WCS
import os
import seaborn as sns
import pandas as pd
import pickle

# specific packages for statistics and principal component analysis
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy import stats
from scipy.optimize import curve_fit

# packages for working with sparse healpix maps
import healsparse as hsp
import skyproj

#packages for working with healpy healpix maps
import healpy as hp

# LSST packages
from lsst.daf.butler import Butler
import lsst.geom as geom

# allow interactive plots
%matplotlib widget

# default plot style is accessible
plt.style.use('tableau-colorblind10')

In [ ]:
config = 'dp02'
collections = '2.2i/runs/DP0.2'
butler = Butler(config, collections=collections)

# 2.0. Display the map of magnitude limit

In [ ]:
hspmap = butler.get('deepCoadd_psf_maglim_consolidated_map_weighted_mean', band='i')

In [ ]:
nside_coverage = hspmap.nside_coverage
nside_sparse = hspmap.nside_sparse
print('nside_coverage = ', hspmap.nside_coverage)
print('nside_sparse = ', hspmap.nside_sparse)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sp = skyproj.McBrydeSkyproj(ax=ax, lon_0=65.0)
sp.draw_hspmap(hspmap, vmin=26.0, vmax=26.3)
sp.draw_colorbar(label='PSF Maglim (i-band)')
plt.show()

del fig, ax, sp

# 3.0. Degrade the SP map to lower nside resolution 

In [ ]:
deg_nside = 512

In [ ]:
hspmap = hspmap.degrade(deg_nside)

In [ ]:
print('nside_coverage = ', hspmap.nside_coverage)
print('nside_sparse = ', hspmap.nside_sparse)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sp = skyproj.McBrydeSkyproj(ax=ax, lon_0=65.0)
sp.draw_hspmap(hspmap, vmin=26.0, vmax=26.3)
sp.draw_colorbar(label='PSF Maglim (i-band)')
plt.show()

del fig, ax, sp

# 4.0. Binning the SP maps on sky 
The idea is to bin the value distribution of the SP map to later evaluate $n_{gal}$ on those bins 
projected on the sky. SP maps with very skewed distributions (with long tails) can result in bins 
with very low statistics (few pixels lying on them) or even empty bins if using an equal width binning. 
Then, it is useful to use an equal area binning, which ensures that each SP bin covers similar areas 
on the sky and therefore contain similar statistics 

In [ ]:
def equal_area_bin_edges(map_data,nbins):
        data_min = map_data.min()
        data_max = map_data.max()
        pix_per_bin = int(len(map_data)/nbins)

        data_sort = np.sort(map_data)
        if nbins*pix_per_bin==len(map_data):
                data_sort = np.append(data_sort,data_max)
        binedges = [data_sort[i*pix_per_bin] for i in range(nbins+1)]
        binedges[-1] = data_max

        if len(np.unique(binedges)) != len(binedges):
                raise RuntimeError('Your bin edges are not unique please set them manually')

        return binedges

In [ ]:
# Get the pixels where there are actual SP measurements 
valid_pix = hspmap.valid_pixels

In [ ]:
# It is also possible to obtain the RA,DEC coordinates of the center of the valid pixels. This is useful 
# for generating additional masks 
sp_ra, sp_dec = hspmap.valid_pixels_pos(lonlat=True)

In [ ]:
# Get the values of the SP map on the valid pixels (otherwise, the pixel contains the sentinel value hp.UNSEEN
vals = hspmap.get_values_pix(valid_pix, nest=True)

In [ ]:
# Basic checks to ensure the outputs make sense 
print(len(valid_pix),len(vals),len(sp_ra))

Let's compute the equal area bins for our reference SP map 

In [ ]:
binedges1d = equal_area_bin_edges(vals,nbins=4)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
_ = plt.hist(vals,bins=100)
for spbin in binedges1d:
    plt.axvline(x=spbin,ls='--',color='orange')
plt.plot([],[],ls='--',color='orange',label='Bin edges')
plt.grid()
plt.yscale("log")
plt.xlabel('SP values')
plt.ylabel('Number of pixels')
plt.legend(loc="upper left")
plt.show()

del fig, ax

To better visualize this process, we can project these bins on the sky 

In [ ]:
sky_bins = np.ones(len(valid_pix))
for ibin in range(len(binedges1d)-1):
    ibin_mask = (vals>binedges1d[ibin])*(vals<binedges1d[ibin+1])
    sky_bins[ibin_mask] = ibin+1

In [ ]:
hsp_bins = hsp.HealSparseMap.make_empty(hspmap.nside_coverage, hspmap.nside_sparse, dtype=np.float64)
hsp_bins.update_values_pix(valid_pix, sky_bins,operation='replace')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sp = skyproj.McBrydeSkyproj(ax=ax, lon_0=65.0)
sp.draw_hspmap(hsp_bins)
sp.draw_colorbar(label='PSF Maglim (i-band)')
plt.show()

del fig, ax, sp

Let's load additional SP maps at nside = 512. In the tutorial 03c_Survey_Property_Maps we can learn how 
to check the available SP maps 

In [ ]:
sp_names = []
for dtype in sorted(butler.registry.queryDatasetTypes(expression="*consolidated_map*")):
    print(dtype.name)
    sp_names.append(dtype.name)

In [ ]:
# Uncomment this cell to run with a smaller set of maps 
'''
sp_names = ['deepCoadd_exposure_time_consolidated_map_sum',
            'deepCoadd_psf_maglim_consolidated_map_weighted_mean',
            'deepCoadd_psf_size_consolidated_map_weighted_mean',
            'deepCoadd_sky_background_consolidated_map_weighted_mean']
'''

In [ ]:
# We create some short names for each SP map 
short_names = {}
for sp_name in sp_names:
    name_ = sp_name.replace('deepCoadd_','')
    if 'weighted_mean' in name_:
        name_ = name_.replace('_consolidated_map_weighted_mean','')
    elif 'sum' in name_:
        name_ = name_.replace('_consolidated_map_sum','')
    short_names[sp_name] = name_

In [ ]:
# Select a photometric band 
band = 'i'

In [ ]:
# Here we generate a dictionary containing necessary SP map information for our next steps. Since this takes a while 
# (a few minutes), you can find the dictionary already saved in npy format in this repository. In case the dictionary 
# is not available, you just need to uncomment this cell and generate it
'''
map_dict = {}
for name in sp_names:
    hspmap_ = butler.get(name, band='i')
    hspmap_ = hspmap_.degrade(deg_nside)
    dict_ = {}
    dict_['nside_coverage'] = hspmap_.nside_coverage
    dict_['nside_sparse'] = hspmap_.nside_sparse
    valid_pixels_ = hspmap_.valid_pixels
    dict_['valid_pixels'] = valid_pixels_
    dict_['map_values'] = np.array(hspmap_.get_values_pix(valid_pixels_, nest=True))
    
    map_dict[name] = dict_
    del hspmap_
    
np.save('data_dict_sp_maps_nside{0}.npy'.format(deg_nside),map_dict)
'''

In [ ]:
map_dict = np.load('data_dict_sp_maps_nside{0}.npy'.format(deg_nside),allow_pickle=True).ravel()[0]

In [ ]:
# We verify that all selected SP maps are defined on the same region of the sky 
for sp in sp_names:
    print((map_dict[sp]['valid_pixels']==valid_pix).all())

In [ ]:
map_dict[sp_names[0]]

# 5.0 Compute the correlation matrix of the SP maps 
It is interesting to evaluate the correlation between SP to better understand them. Moreover, one of the 
main purposes of these maps is to correct for their impact on the data, so it is useful to look at the 
correlations in order to have an idea of whether there are maps than can be excluded 

# 5.1 Compute correlation matrix based on Pearson's correlation coefficient 

In [ ]:
corr_matrix_p = np.zeros((len(sp_names),len(sp_names)))
for i,map_i in enumerate(sp_names):
    vals_i = np.array(map_dict[map_i]['map_values'])
    for j,map_j in enumerate(sp_names):
        vals_j = map_dict[map_j]['map_values']
        corr_matrix_p[i,j] = stats.pearsonr(vals_i,vals_j)[0]
print(corr_matrix_p)

# 5.2 Compute correlation matrix based on Spearman's correlation coefficient 
Pearson's correlation coefficient assumes that the random variables have a linear relation between them. 
We can check this assumption computing the Spearman's correlation coefficient, which only assumes a 
correlation given by a monotonic function (not necessarily a linear one). If Pearson's and Spearman's 
coefficients are close, that is a sign of linear correlation 

In [ ]:
corr_matrix_s = np.zeros((len(sp_names),len(sp_names)))
for i,map_i in enumerate(sp_names):
    vals_i = np.array(map_dict[map_i]['map_values'])
    for j,map_j in enumerate(sp_names):
        vals_j = map_dict[map_j]['map_values']
        corr_matrix_s[i,j] = stats.spearmanr(vals_i,vals_j)[0]
print(corr_matrix_s)

We can plot now the correlation matrices 

In [ ]:
xlabel_matrix = []
ylabel_matrix = []
for i,sp_name in enumerate(sp_names):
    xlabel_matrix.append(short_names[sp_name]+' ({0})'.format(i))
    ylabel_matrix.append(str(i))

In [ ]:
fig = plt.figure(figsize=(8.5,8.5))
ax = fig.add_subplot(1,1,1)
sns.heatmap(corr_matrix_p,vmin=-1.0,vmax=1.0,annot=True,cmap='jet',linewidth=0.5,square=True,cbar=True,xticklabels=xlabel_matrix,yticklabels=ylabel_matrix)
plt.title(r'$r_P$ coeff, band-{0}'.format(band))

del fig, ax

In [ ]:
fig = plt.figure(figsize=(8.5,8.5))
ax = fig.add_subplot(1,1,1)
sns.heatmap(corr_matrix_s,vmin=-1.0,vmax=1.0,annot=True,cmap='jet',linewidth=0.5,square=True,cbar=True,xticklabels=xlabel_matrix,yticklabels=ylabel_matrix)
plt.title(r'$r_S$ coeff, band-{0}'.format(band))

del fig, ax

# 6.0. Do principal component analysis (PCA) of the SP maps 

In [ ]:
def pca_summary(pca, standardised_data, out=True):
        names = ["PC"+str(i) for i in range(1, len(pca.explained_variance_ratio_)+1)]
        a = list(np.std(pca.transform(standardised_data), axis=0))
        b = list(pca.explained_variance_ratio_)
        c = [np.sum(pca.explained_variance_ratio_[:i]) for i in range(1,len(pca.explained_variance_ratio_)+1)]
        columns = pd.MultiIndex.from_tuples([("sdev", "Standard deviation"), ("varprop", "Proportion of Variance"), ("cumprop", "Cumulative Proportion")])
        summary = pd.DataFrame(zip(a, b, c), index=names, columns=columns)
        if out:
                print("Importance of components:")
                display(summary)
        return summary

In [ ]:
def screeplot(pca, standardised_values, figsize):
        y = np.std(pca.transform(standardised_values), axis=0)**2
        x = np.arange(len(y)) + 1
        fig = plt.figure(figsize=figsize)
        fig.set_tight_layout(True)
        plt.plot(x, y, "o-")
        plt.xticks(x, ["Comp."+str(i) for i in x], ha='right', rotation=50, fontsize=12)
        plt.ylabel("Variance")
        plt.grid()
        plt.show()
        #plt.close()

In [ ]:
data_dict = {}
for key in map_dict:
    data_dict[key] = map_dict[key]['map_values']

In [ ]:
dataframe = pd.DataFrame(data_dict)

In [ ]:
standard = scale(dataframe)
standard_df = pd.DataFrame(standard,columns=dataframe.columns)
pca = PCA().fit(standard_df)

In [ ]:
summary = pca_summary(pca, standard_df)

In [ ]:
summary.to_csv('data_frame.csv',header=True)

In [ ]:
screeplot(pca, standard_df, figsize=(8,6))

In [ ]:
# In this matrix, each row corresponds to each PC and the values in that row correspond to the coefficients 
# each original SP map should be multiplied by in order to get that PC map 
print(pca.components_)

In [ ]:
print(pca.components_[0])

In [ ]:
# Check that the sum of the squared coefficients for each PC map is 1 
print(np.sum(pca.components_[0]**2.))

In [ ]:
for i in range(len(sp_names)):
        pcavalues = pca.transform(standard_df)[:,i]
        
        hsp_pca_ = hsp.HealSparseMap.make_empty(hspmap.nside_coverage, deg_nside, dtype=np.float64)
        hsp_pca_.update_values_pix(valid_pix, pcavalues,operation='replace')
        
        assert (hsp_pca_.valid_pixels==valid_pix).all()
        
        # Let's plot some PC maps on the sky 
        if i<2 or i>len(sp_names)-3:
            print('PC {0}'.format(i+1))
            fig, ax = plt.subplots(figsize=(8, 5))
            sp = skyproj.McBrydeSkyproj(ax=ax, lon_0=65.0)
            sp.draw_hspmap(hsp_pca_)
            sp.draw_colorbar(label='PSF Maglim (i-band)')
            plt.show()
            
            del fig, ax, sp

# 7.0. Load galaxies from dp02_dc2_catalogs.Object and create number galaxy map 
We previously saved the colums that we are interested in from dp02_dc2_catalogs.Object in a pickle file


In [ ]:
def cat2map(ra,dec,nside,weight=None):
        theta = np.radians(90.-dec)
        phi = np.radians(ra)
        
        map1 = np.zeros(hp.nside2npix(nside))
        p = hp.ang2pix(nside,theta,phi,nest=True)
        #print(p)
        mask = np.zeros(hp.nside2npix(nside)).astype('bool')
        mask[p] = True
        if weight is None:
                for i in p:
                        map1[i] += 1
        else:
                for index,i in enumerate(p):
                        map1[i] += weight[index]
        map1[~mask] = hp.UNSEEN
        pix_ra, pix_dec = hp.pix2ang(nside,np.unique(p),nest=True,lonlat=True)
        
        return map1, mask, np.unique(p), pix_ra, pix_dec

In [ ]:
with open('/scratch/mrmonroy/survpropmap/sources_result.pkl','rb') as f:
    cat_table = pickle.load(f)

In [ ]:
cat_table

In [ ]:
ra = cat_table['coord_ra']
dec = cat_table['coord_dec']
refext = cat_table['refExtendedness']

In [ ]:
sel_mask = (refext==1.0)

In [ ]:
sel_ra = np.array(ra[sel_mask])
sel_dec = np.array(dec[sel_mask])

In [ ]:
print(len(sel_ra))
print(sel_ra.min(),sel_ra.max())
print(sel_dec.min(),sel_dec.max())

In [ ]:
ngal_vals, mask, gal_pixels, pix_ra, pix_dec = cat2map(sel_ra,sel_dec,deg_nside)

In [ ]:
assert (np.sum(ngal_vals[mask])==len(sel_ra))

We can have a look at the galaxy number density distribution 

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
n,bins,_ = ax.hist(ngal_vals[mask],bins=100)
ax.grid()
ax.set_xlabel(r'$n_{gal}$')
ax.set_ylabel('Number if pixels')

del fig, ax

Using healSparse we can create a heakpix format map which contains the number of galaxies per pixel 

In [ ]:
hsp_map_ngal = hsp.HealSparseMap.make_empty(nside_coverage, deg_nside, dtype=np.float64)
pixels_ngal = hp.ang2pix(deg_nside, np.radians(90. - pix_dec), np.radians(pix_ra), nest=True)
assert (len(np.unique(pixels_ngal))==len(pixels_ngal))
hsp_map_ngal.update_values_pix(pixels_ngal, ngal_vals[mask],operation='replace')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sp = skyproj.McBrydeSkyproj(ax=ax, lon_0=65.0)
sp.draw_hspmap(hsp_map_ngal)
sp.draw_colorbar(label='ngal')
plt.show()

del fig, ax, sp

In [ ]:
(hsp_map_ngal.get_values_pix(gal_pixels)==ngal_vals[mask]).all()

Now we evaluate the SP maps in the same regions where the $n_{gal}$ map is defined 

In [ ]:
hspmap_masked = hsp.HealSparseMap.make_empty(nside_coverage, deg_nside, dtype=np.float64)
hspmap_masked.update_values_pix(gal_pixels, hspmap.get_values_pix(gal_pixels, nest=True),operation='replace')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sp = skyproj.McBrydeSkyproj(ax=ax, lon_0=65.0)
sp.draw_hspmap(hspmap_masked, vmin=26.0, vmax=26.3)
sp.draw_colorbar(label='PSF Maglim (i-band)')
plt.show()

del fig, ax, sp

Let's mask all the SP maps in preparation for our next step 

In [ ]:
# Again, this can take a few minutes, so you can find the dictionary with the necessary information in the repository. 
# If not, you just need to uncomment this cell and run
'''
masked_map_dict = {}
for name in sp_names:
    hspmap_ = butler.get(name, band='i')
    hspmap_ = hspmap_.degrade(deg_nside)
    assert (hspmap_.nside_sparse==hsp_map_ngal.nside_sparse)
    
    dict_ = {}
    dict_['nside_coverage'] = hspmap_.nside_coverage
    dict_['nside_sparse'] = hspmap_.nside_sparse
    dict_['map_values'] = np.array(hspmap_.get_values_pix(gal_pixels, nest=True))
    
    masked_map_dict[name] = dict_
    del hspmap_
    
np.save('data_dict_masked_sp_maps_nside{0}.npy'.format(deg_nside),masked_map_dict)
'''

In [ ]:
masked_map_dict = np.load('data_dict_masked_sp_maps_nside{0}.npy'.format(deg_nside),allow_pickle=True).ravel()[0]

# 8.0. Compute 1D relations 
We define the function to compute them 

In [ ]:
def bin1d_coords(ngal_map,mask,sp_map_vals,nbins1d):
    ngal_vals = ngal_map[mask]
    ngal_footprint = np.average(ngal_vals)
    
    binedges1d = equal_area_bin_edges(sp_map_vals,nbins=nbins1d)
    
    sp_in_bin1d = []
    ngal_in_bin1d = []
    err_in_bin1d = []
    for ibin in range(len(binedges1d)-1):
        ibin_mask = (sp_map_vals>binedges1d[ibin])*(sp_map_vals<binedges1d[ibin+1])
        
        sp_in_bin1d_ = np.average(sp_map_vals[ibin_mask])
        ngal_in_bin1d_ = np.average(ngal_vals[ibin_mask])
        err_in_bin1d_ = np.std(ngal_vals[ibin_mask])/np.sqrt(len(ngal_vals[ibin_mask]))
        
        sp_in_bin1d.append(sp_in_bin1d_)
        ngal_in_bin1d.append(ngal_in_bin1d_)
        err_in_bin1d.append(err_in_bin1d_)
    
    sp_in_bin1d = np.array(sp_in_bin1d)
    ngal_in_bin1d = np.array(ngal_in_bin1d)
    err_in_bin1d = np.array(err_in_bin1d)
    
    return sp_in_bin1d, ngal_in_bin1d, err_in_bin1d, ngal_footprint
    

In [ ]:
def fun_fit(x,a,b,c):
    return a*x**2.+b*x+c

Let's plot an example 

In [ ]:
nbins1d = 10

In [ ]:
sp_x,ngal_y,ngal_err,ngal_mean = bin1d_coords(ngal_vals,mask,masked_map_dict[sp_names[0]]['map_values'],nbins1d=nbins1d)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.axhline(y=1.0,ls='--',color='b')
ax.plot(sp_x,ngal_y/ngal_mean,color='r')
plt.errorbar(sp_x,ngal_y/ngal_mean,yerr=ngal_err/ngal_mean,fmt='.',color='r')
ax.grid()
ax.set_xlabel(sp_names[0])
plt.ylabel(r'$n_{gal}/\langle n_{gal} \rangle$')
plt.show()

Now let's do this for all SP maps and fit each 1D relation with a quadratic function 

In [ ]:
dict_1d = {}
for name in sp_names:
    dict_1d_ = {}
    
    sp_x_,ngal_y_,ngal_err_,ngal_mean_ = bin1d_coords(ngal_vals,mask,masked_map_dict[name]['map_values'],nbins1d=nbins1d)
    
    p0,cov0 = curve_fit(fun_fit,sp_x_,ngal_y_/ngal_mean_,sigma=ngal_err_/ngal_mean_)
    ngal_fit_ = fun_fit(sp_x_,p0[0],p0[1],p0[2])
    
    dict_1d_['x'] = sp_x_
    dict_1d_['y'] = ngal_y_
    dict_1d_['err'] = ngal_err_
    dict_1d_['ngal_mean'] = ngal_mean_
    dict_1d_['yfit'] = ngal_fit_
    dict_1d_['fit_coeffs'] = p0
    
    dict_1d[name] = dict_1d_

In [ ]:
dict_1d.keys()

In [ ]:
sp_names

In [ ]:
num_columns = 2
num_rows = int(len(sp_names)/num_columns)+int(len(sp_names)%num_columns)

fig, axs = plt.subplots(num_rows, num_columns, figsize=(8,14))

i = 0
for row in range(num_rows):
    for col in range(num_columns):
        if i<len(sp_names):
            sp_name = sp_names[i]
            i += 1
                
            ngal_mean_ = dict_1d[sp_name]['ngal_mean']
            x_ = dict_1d[sp_name]['x']
            y_ = dict_1d[sp_name]['y']/ngal_mean_
            err_ = dict_1d[sp_name]['err']/ngal_mean_
            yfit_ = dict_1d[sp_name]['yfit']
                
            axs[row,col].axhline(y=1.0,ls='--',color='b')
            axs[row,col].plot(x_,y_,color='r')
            axs[row,col].errorbar(x_,y_,yerr=err_,fmt='.',color='r')
            axs[row,col].plot(x_,yfit_,ls='--',color='purple')
            axs[row,col].grid()
            axs[row,col].set_xlabel('SP value')
            axs[row,col].set_ylabel(r'$n_{gal}/\langle n_{gal} \rangle$')
            axs[row,col].set_title(sp_name, fontsize='8')
        
plt.tight_layout()
plt.show()
del fig, axs

# 9.0 Derive a weight map from the 1D fits 
Ideally, we should compute the error bars and the covariance between 1D bins using simulations (computing the 
1D relation of each SP map with each simulation) or with other techniques, such as Jackknife or Bootstrap. Here, 
for simplicity, the error bar of each 1D bin is obtained as $\sigma_{n_{gal}}(bin)/\sqrt N$, with $N$ the number of 
pixels in each 1D bin (given the equal area binning, this number is approximately the same for all bins) 

In [ ]:
weight_sp_name = 'deepCoadd_exposure_time_consolidated_map_sum'
weight_sp = masked_map_dict[weight_sp_name]['map_values']
a_sp = dict_1d[weight_sp_name]['fit_coeffs'][0]
b_sp = dict_1d[weight_sp_name]['fit_coeffs'][1]
c_sp = dict_1d[weight_sp_name]['fit_coeffs'][2]

In [ ]:
w_map = 1./fun_fit(weight_sp,a_sp,b_sp,c_sp)

We can plot the value distribution of the weight map 

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
n,bins,_ = ax.hist(w_map,bins=100)
ax.grid()
ax.set_xlabel(r'$w$')
ax.set_ylabel('Number if pixels')

del fig, ax

Now, we apply the weight map to the $n_{gal}$ map and recompute the 1D relation 

In [ ]:
ngal_w = np.copy(ngal_vals)
ngal_w[mask] = ngal_vals[mask]*w_map

In [ ]:
sp_w_x,ngal_w_y,ngal_w_err,ngal_mean_w = bin1d_coords(ngal_w,mask,weight_sp,nbins1d=nbins1d)

In [ ]:
# Load the 1D coordinates from before applying weights 
ngal_mean = dict_1d[weight_sp_name]['ngal_mean']
sp_x = dict_1d[weight_sp_name]['x']
ngal_y = dict_1d[weight_sp_name]['y']
ngal_err = dict_1d[weight_sp_name]['err']

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.axhline(y=1.0,ls='--',color='b')
ax.plot(sp_x,ngal_y/ngal_mean,color='r',label='Unweighted data')
plt.errorbar(sp_x,ngal_y/ngal_mean,yerr=ngal_err/ngal_mean,fmt='.',color='r')
ax.plot(sp_w_x,ngal_w_y/ngal_mean_w,color='b',label='Weighted data')
plt.errorbar(sp_w_x,ngal_w_y/ngal_mean_w,yerr=ngal_w_err/ngal_mean_w,fmt='.',color='b')
ax.grid()
ax.set_xlabel(weight_sp_name)
ax.set_ylabel(r'$n_{gal}/\langle n_{gal} \rangle$')
ax.legend(loc="lower right")
plt.show()

We can see in blue how the impact of this particular SP map is mitigated by applying its corresponding weight map 